<a href="https://colab.research.google.com/github/lescarpi/pdi-2025/blob/main/Roteiro_PDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🍎 Identificação da Qualidade de Frutas

📝 Introdução

Este notebook tem como objetivo demonstrar como técnicas de processamento de
imagens podem ser combinadas com modelos de Machine Learning para detecção de
alimentos estragados, neste caso, com foco específico em maçãs.

⚙ Método

Utilizaremos um dataset com imagens de maçãs saudáveis e podres, cada uma
das imagens será processada com as seguintes técnicas:
- Filtro Gaussiano para suavização e diminuição de imperfeições.
- CLAHE para melhoramento do contraste.
- Erosão seguida de dilatação para remover ruídos.

Após isso será feito o treinamento do modelo de Rede Neural Convolucional (CNN)
utilizando a biblioteca Keras.

💻 Usabilidade

O usuário participará de um jogo que consiste na apresentação de 15 imagens
de maçãs e o mesmo terá que responder se as frutas estão saudáveis ou não,
em seguida será demonstrado a eficiência do modelo para as mesmas imagens.

Não é necessário rodar todos os trechos de código para a competição interativa
com o modelo, caso queira pular, rode apenas a partir do penúltimo trecho de código, que carrega o modelo.

Vamos iniciar instalando a API do Kaggle para fazer o download do dataset:

In [ ]:
!pip install kaggle

Configuração das credenciais necessárias para uso da API

In [ ]:
import os
import json

# Criação do diretório de configuração do Kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

# Credenciais
kaggle_token = {
    "username": "lescarpi",
    "key": "ca3028ff182e37ea32f5552a65b37a5e"
}

# Salvando as credenciais como um arquivo JSON
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(kaggle_token, file)

# Ajustando permissões
os.chmod('/root/.kaggle/kaggle.json', 0o600)

Realizando autenticação e baixando o dataset

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Criando diretório se não existir
dataset_path = "/content/dataset"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

api = KaggleApi()
api.authenticate()

api.dataset_download_files('lescarpi/apple-disease-dataset', path=dataset_path, unzip=True)

# Verificando os arquivos baixados
print(os.listdir(dataset_path))

Dataset URL: https://www.kaggle.com/datasets/lescarpi/apple-disease-dataset
['Apples Disease Dataset', 'Apples Disease Processed Dataset']


Com as imagens baixadas, vamos definir as funções que serão utilizadas para processamento

In [ ]:
!pip install opencv-python

Definindo algumas funções de processamento de imagem que serão utilizadas

In [ ]:
import cv2
import numpy as np

def aplicar_filtro_gaussiano(imagem):
    return cv2.GaussianBlur(imagem, (5, 5), 0)

def aplicar_clahe(imagem):
    lab = cv2.cvtColor(imagem, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    lab_clahe = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def aplicar_erosao(imagem):
    kernel = np.ones((3, 3), np.uint8)
    return cv2.erode(imagem, kernel, iterations=1)

def aplicar_dilatacao(imagem):
    kernel = np.ones((3, 3), np.uint8)
    return cv2.dilate(imagem, kernel, iterations=1)

def tratar_imagem(imagem):
    imagem = aplicar_filtro_gaussiano(imagem)
    imagem = aplicar_clahe(imagem)
    imagem = aplicar_erosao(imagem)
    imagem = aplicar_dilatacao(imagem)
    return imagem


Agora vamos processar todas as imagens e salvar em uma nova pasta, aqui passamos todas as imagens de treino por esses filtros

In [ ]:
def processar_pastas(origem_base, destino_base):
    categorias = ['healthy', 'rotten']

    for categoria in categorias:
        origem = os.path.join(origem_base, categoria)
        destino = os.path.join(destino_base, categoria)
        os.makedirs(destino, exist_ok=True)

        for nome_arquivo in os.listdir(origem):
            caminho_imagem = os.path.join(origem, nome_arquivo)
            imagem = cv2.imread(caminho_imagem)

            if imagem is None:
                print(f"Não foi possível carregar {caminho_imagem}")
                continue

            imagem_tratada = tratar_imagem(imagem)

            caminho_destino = os.path.join(destino, nome_arquivo)
            cv2.imwrite(caminho_destino, imagem_tratada)

base = dataset_path + "/" "Apples Disease Dataset"
destino = dataset_path + "/" + "Apples Disease Processed Dataset"

processar_pastas(base, destino)

Importando bibliotecas de Machine Learning e configurando

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Caminho para o dataset processado com as pastas 'healthy' e 'rotten'
train_dir = destino

# Usando ImageDataGenerator para carregar as imagens e realizar aumento de dados
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalizar as imagens
    rotation_range=20,         # Rotacionar as imagens aleatoriamente
    width_shift_range=0.2,     # Deslocar aleatoriamente a imagem horizontalmente
    height_shift_range=0.2,    # Deslocar aleatoriamente a imagem verticalmente
    shear_range=0.2,           # Aplicar cisalhamento
    zoom_range=0.2,            # Aplicar zoom
    horizontal_flip=True,      # Girar imagens horizontalmente
    fill_mode='nearest'        # Preencher os espaços vazios com o valor mais próximo
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),    # Redimensionar para o tamanho esperado pelo modelo
    batch_size=32,
    class_mode='binary'        # Como temos 2 classes: saudável e estragada
)

Found 5363 images belonging to 2 classes.


Criando modelo de Rede Neural Convolucional (CNN), treinando e salvando

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Saída binária: saudável ou estragada
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(train_generator, epochs=15, verbose=1)

Epoch 1/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.6020 - loss: 0.6986
Epoch 2/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.8426 - loss: 0.3627
Epoch 3/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.8914 - loss: 0.2700
Epoch 4/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.9022 - loss: 0.2355
Epoch 5/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 312s 2s/step - accuracy: 0.9073 - loss: 0.2124
Epoch 6/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.9222 - loss: 0.1995
Epoch 7/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.9399 - loss: 0.1431
Epoch 8/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.9394 - loss: 0.1495
Epoch 9/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.9509 - loss: 0.1234
Epoch 10/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 319s 2s/step - accuracy: 0.9455 - loss: 0.1341
Epoch 11/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 302s 2s/step - accuracy: 0.9580 - loss: 0.1100
Epoch 12/15
168/168 ━━━━━━━━━━━━━━━━━━━━ 

Salvando o modelo no Google Drive para não precisar treinar de novo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Salvar modelo no Drive
model.save('/content/drive/MyDrive/model__preprocessed.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Caregando o modelo já treinado

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/model__preprocessed.h5')

Mounted at /content/drive


O trecho a seguir abrirá uma interface para uma competição interativa, responda se considera que a maçã apresentada está boa para consumo ou não

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import ipywidgets as widgets
from IPython.display import display, clear_output

# Caminho da pasta
folder_path = '/content/drive/MyDrive/fotos_macas/'

def prepare_image(image):
    image = tratar_imagem(image)
    image = Image.fromarray(image)
    image = image.convert('RGB')
    image = image.resize((150, 150))
    image = np.array(image) / 255.0
    return np.expand_dims(image, axis=0)

def predict(image):
    image = prepare_image(image)
    prediction = model.predict(image)
    pred_value = prediction[0][0]
    if pred_value < 0.5:
        return "Saudável", (1 - pred_value)
    else:
        return "Estragada", pred_value

# Lista e embaralha imagens
image_files = [f for f in os.listdir(folder_path) if f.endswith(('jpg', 'png', 'jpeg'))]
random.shuffle(image_files)

# Variáveis de controle
current_index = 0
acertos = 0
current_image_array = None  # Para garantir que a predição e visualização batem

# Widgets
output = widgets.Output()
btn_saudavel = widgets.Button(description="🍏 Saudável", button_style='success')
btn_estragada = widgets.Button(description="☠️ Estragada", button_style='danger')
btn_proxima = widgets.Button(description="Próxima ➡️", button_style='info', disabled=True)
buttons_resposta = widgets.HBox([btn_saudavel, btn_estragada])

# Função para mostrar imagem
def mostrar_imagem():
    global current_index, current_image_array
    output.clear_output(wait=True)
    if current_index < len(image_files):
        image_path = os.path.join(folder_path, image_files[current_index])
        image = cv2.imread(image_path)
        if image is None:
            print(f"Erro ao carregar imagem: {image_path}")
            return
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        current_image_array = image.copy()  # Guardar para predição
        with output:
            plt.imshow(image)
            plt.axis('off')
            plt.show()
        btn_proxima.disabled = True
        btn_saudavel.disabled = False
        btn_estragada.disabled = False
    else:
        with output:
            print(f"\n🎉 Fim do jogo! Você acertou {acertos} de {len(image_files)}.")
        buttons_resposta.layout.display = 'none'
        btn_proxima.layout.display = 'none'

# Função para tratar clique em saudavel ou estragada
def on_click_resposta(opcao):
    def handler(b):
        global current_index, acertos
        label, prob = predict(current_image_array)

        output.clear_output(wait=True)
        with output:
            plt.imshow(current_image_array)
            plt.axis('off')
            plt.show()
            print(f"Você escolheu: {opcao}")
            print(f"Resultado do modelo: {label}")

            if opcao == label:
                print("✅ Sua resposta condiz com a do modelo!\n")
                acertos += 1
            else:
                print("❌ O modelo avaliou o contrário...\n")

        btn_proxima.disabled = False
        btn_saudavel.disabled = True
        btn_estragada.disabled = True
    return handler

def on_click_proxima(b):
    global current_index
    current_index += 1
    mostrar_imagem()

# Associar handlers
btn_saudavel.on_click(on_click_resposta("Saudável"))
btn_estragada.on_click(on_click_resposta("Estragada"))
btn_proxima.on_click(on_click_proxima)

# Mostrar interface
display(buttons_resposta)
display(btn_proxima)
display(output)
mostrar_imagem()

Button(button_style='info', description='Próxima ➡️', disabled=True, style=ButtonStyle())

Output()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


💡 Conclusão

A competição interativa com o modelo demonstra, na prática, o potencial de um
modelo preditivo para avaliar qualidade de alimentos, as técnicas de processamento digital de imagens colaboram muito com o aprendizado, padronizando
e melhorando as imagens.